In [1]:
#For a prime power q and a number e,
#this program computes all chain rings with residue field k
#and ramification index e (i.e. (p)=m^e),
#up to isomorphism respecting the augmentation.
#Every chain ring R is represented by its length n and a choice of
#Eisenstein polynomial E, with R=W(k)[z]/(E(z),z^n).
#This representation is of course not unique and depends on a choice
#of uniformizer. Our conventions are such that the chosen uniformizer
#for a given tree always maps to the chosen uniformizers
#for all of its quotients. In particular, if two chain rings R, R' have
#isomorphic length n quotients R/m^n and R'/m^n, with our conventions
#their Eisenstein polynomials will agree modulo valuation n/e, i.e.
#the coefficient in front of z^i will agree modulo p^((n-i)//e)

In [2]:
def nextDisks(R,n,disks,z):
    pi = R.uniformiser()
    newDisks = []
    for disk in disks:
        polynomialReduced = disk["polynomial"].change_ring(R.residue_field())
        #other case: polynomialReduced == 0
        if polynomialReduced.is_zero():
            newdisk = dict(center = disk["center"], polynomial = disk["polynomial"]/pi, size=disk["size"])
            newDisks.append(newdisk)
        else:     
            for root,_ in polynomialReduced.roots():
                rootlift = R(root).lift_to_precision()
                newpoly = disk["polynomial"](pi*z+rootlift)/pi
                newcenter = disk["center"]+rootlift*pi**disk["size"]
                newdisk = dict(center = newcenter, polynomial = newpoly, size=disk["size"]+1)
                newDisks.append(newdisk)
    return newDisks


def findApproximateZeros(R,f,n):
    pi = R.uniformiser()
    Rpoly.<z> = R[]
    f2 = f(z)
    disks = [dict(
        center=0,
        polynomial=f2,
        size=0,
    )]
    for precision in range(n):
        disks = nextDisks(R,precision,disks,z)
    return disks


#chain ring specified relative base unramified extension W


def chainRingMass(W,E,n):
    if n<=E.degree():
        return 1
    R.<pi> = W.extension(E)
    q = W.residue_field().cardinality()
    aut = 0
    for disk in findApproximateZeros(R,E,n):
        aut = aut + q^(n-disk["size"])
    return 1/aut

def euclidTransforms(F,transforms):
    x = F.gen()
    q = F.order()
    translations = []
    pivot = None
    while len(transforms)>0:
        otherTransforms = []
        #print("in while loop")
        for shift,log in transforms:
            if log == 0:
                translations.append(shift)
            elif pivot == None:
                pivot = (shift,log)
            else:
                quot = log // pivot[1]
                newLog = log % pivot[1]
                power = x^((quot * pivot[1]) % (q-1))
                newShift = shift - x^log * pivot[0] * (power - 1) / (power * (x^pivot[1]))
                if newLog == 0:
                    translations.append(newShift)
                else:
                    otherTransforms.append(pivot)
                    pivot = (newShift,newLog)
        transforms = otherTransforms
    return pivot,translations


def initial(W,e):
    #Chain rings of ramification e and length e+1 are given by E(z) = z^e + ap,
    #where a ranges over equivalence classes of k^x / (k^x)^e
    Wpoly.<z> = W[]
    F = R.residue_field()
    x = F.primitive_element()
    p = F.characteristic()
    q = F.order()
    d = gcd(q-1,e)
    return [(z^e + W(x^i).lift_to_precision()*p,e+1) for i in range(d)]

def branches(W,E,n):
    #for each automorphism f of W[z]/(E(z),z^n), have well-defined a in k with
    #E(f(z)) = a z^n mod (E(z),z^{n+1}).
    #This defines map Aut(E,n) -> k \semidirect (k^\times)^n
    #For each disk in findApproximateZeros(R,E,n), the image of the automorphism
    #corresponding to pi -> disk["center"] + pi^k*t is
    # (R.residue_field(disk["poly"](t)),R.residue_field(disk["center"]/pi)^n)
    # group is generated by images of pi -> disk["center"] and pi -> pi + c*pi^k where c generate residue field additively.
    e = E.degree()
    if(n<=e):
        raise Exception("length needs to be bigger than ramification")
    R.<pi> = W.extension(E)
    F = R.residue_field()
    p = F.characteristic()
    q = F.order()
    z = E.variables()[0]

    disks = findApproximateZeros(R,E,n)
    
    #at some point, Hensel lifting guarantees a unique zero to E
    #we detect this directly below, but it would suffice to have n > 2*e*val(E'(pi)).
    # val(E'(pi)) is at most val(e) + (e-1)/e, so n > 2e(val(e)+1) - 2 suffices.
    if(disks[0]["polynomial"].change_ring(R.residue_field()).degree() == 1):
        return "stable"
    
    #Here we compute the image in the group of affine linear transformations
    # k \semidirect (k^\times)^n described above.
    transforms = []
    for disk in disks:
        #print(disk)
        if(disk["center"].is_equal_to(pi,disk["size"])):
            x = F.primitive_element()
            for i in range(F.degree()):
                #print((disk["polynomial"].change_ring(F)(x**i),1))
                transforms.append((disk["polynomial"].change_ring(F)(x**i),0))
        else:
            #print((F(disk["polynomial"](0)),F((disk["center"]/pi)^n)))
            transforms.append((F(disk["polynomial"](0)),F((disk["center"]/pi)^n).log(F.gen())))
            
    #Now we apply a version of euclid's algorithm (implemented above) to bring our generators
    # into the form "list of translations + at most one affine-linear map which is not a translation"
    pivot,translations = euclidTransforms(F,transforms)
    #print(f"pivot: {pivot}")
    #print(f"translations: {translations}")
    
    #The rotation coefficient of the remaining map generates a smallest subfield L in our residue field F,
    # and the subgroup of all translations is the sub L vector space generated by our translations.
    # Also, the rotation fixes a unique element, which we will use later to reindex orbits.
    subfield_deg = 1
    center_of_pivot = 0
    if not (pivot == None):
        shift,log = pivot
        subfield_deg = Mod(p,(q-1)/log).multiplicative_order()
        center_of_pivot = shift / (1 - F.gen()^log)
    #print(f"degree of subfield: {subfield_deg}")
    L = F.subfield(subfield_deg)
    
    #We now view F as L vector space and pick a complement for the sub vector space generated by our translations
    V,iso1,iso2 = F.free_module(map=True,base=L)
    V_comp = V.subspace([iso2(sh) for sh in translations]).complement()
    
    #For a basis bi of V_comp, the list of representative of a's is given by
    #center_of_pivot + lam1*b1 + ... + lamn*bn,
    #up to multiplication by x^log (the rotational coefficient of pivot)
    cosets = []
    
    if not (pivot == None):
        bs = [iso1(b) for b in V_comp.basis()]
        coeffCosets = [coset[0] for coset in L.cyclotomic_cosets(L(F.gen()^log))]
        allVectors = [0]
        cosets = [center_of_pivot]
        for i in range(len(bs)-1,-1,-1):
            cosets = cosets + [center_of_pivot + lam * bs[i] + vec for lam in coeffCosets if not lam == 0 for vec in allVectors]
            if i>0:
                newAllVectors = [lam * bs[i] + vec for lam in L for vec in allVectors]
                allVectors = newAllVectors
    else:
        cosets = [iso1(v) for v in V_comp]
    
    #now, for each coset representative a, the corresponding Eisenstein is
    # E(z) + a*(-E(0))^(n//e)*z^(n%e)
    
    return [(E + W(a).lift_to_precision()*(-E(0))^(n//e)*z^(n % e),(n+1)) for a in cosets]

def tree(W,E,n):
    bs = branches(W,E,n)
    if bs == "stable":
        node = dict(
            mass = chainRingMass(W,E,n),
            eisenstein = E,
            children = []
        )
        return node    
    else:
        node = dict(
            mass = chainRingMass(W,E,n),
            eisenstein = E,
            children = [tree(W,E2,n2) for E2,n2 in bs]
        )
        return node

def classifyChainrings(W,e):
    F = W.residue_field()
    q = F.order()
    root = dict(
        mass = q^(1-e),
        children = [tree(W,E,n) for E,n in initial(W,e)]
    )
    return root

In [3]:
def testClassification(tree):
    if (tree["children"] == []):
        return True
    else:
        childMass = sum([ child["mass"] for child in tree["children"]])
        childTest = all([testClassification(child) for child in tree["children"]])
        return (childMass == tree["mass"] and childTest)
    
def drawClassification(tree):
    g = DiGraph({},loops=True)
    free=dict(id=1)
    def addTree(tr,free):        
        node = free["id"]
        #print(node)
        free["id"] = free["id"] + 1
        if(tr["children"]==[]):
            leafId = tr["eisenstein"].change_ring(IntegerModRing(64))
            g.add_edge(node,leafId)
            #print(f"adding leaf edge from {node} to {leafId}")
        
        else:
            for child in tr["children"]:
                childId = addTree(child,free)
                #print(f"adding inner edge from {node} to {childId}")
                g.add_edge(node,childId)
        return node
    addTree(tree,free)
    g.plot(edge_style='solid', layout='tree').show(figsize=(8,8))

In [6]:
W.<x> = Zq(2)
e=2

In [7]:
classifyChainrings(W,e)

AttributeError: type object 'R' has no attribute 'residue_field'

In [8]:
drawClassification(classifyChainrings(W,e))

AttributeError: type object 'R' has no attribute 'residue_field'